In [1]:
!git clone https://github.com/Yichuan0712/11785-TCR.git
import os
print(os.getcwd())
os.chdir('11785-TCR')
# os.listdir(os.getcwd())

Cloning into '11785-TCR'...
remote: Enumerating objects: 1027, done.
remote: Counting objects: 100% (237/237), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 1027 (delta 144), reused 218 (delta 125), pack-reused 790 (from 1)
Receiving objects: 100% (1027/1027), 2.53 MiB | 5.70 MiB/s, done.
Resolving deltas: 100% (580/580), done.
/content


In [2]:
!pip install 'git+https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup'
!pip install 'git+https://github.com/facebookresearch/esm.git'
!pip install peft

  Cloning https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup to /tmp/pip-req-build-yvwdhtgh
  Running command git clone --filter=blob:none --quiet https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup /tmp/pip-req-build-yvwdhtgh
  Resolved https://github.com/katsura-jp/pytorch-cosine-annealing-with-warmup to commit 12d03c07553aedd3d9e9155e2b3e31ce8c64081a
  Preparing metadata (setup.py) ... done
  Created wheel for cosine_annealing_warmup: filename=cosine_annealing_warmup-2.0-py3-none-any.whl size=4167 sha256=83ed1e202675ae95e8a97124954e0875f5645d6036493e1ddd0153307364ba90
  Stored in directory: /tmp/pip-ephem-wheel-cache-h8u5jro8/wheels/29/26/10/bf1a07417dd54aa73bdf09ce4f31c187974a444a1cedddbd99
Successfully built cosine_annealing_warmup
  Cloning https://github.com/facebookresearch/esm.git to /tmp/pip-req-build-18nb_v_k
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/esm.git /tmp/pip-req-build-18nb_v_k
  Resolved

In [3]:
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import OneHotEncoder

df = pd.read_csv('/content/11785-TCR/feature_data_test.csv')

In [4]:
df2 = pd.read_csv('/content/11785-TCR/dataset/pytdc/test_PyTDC.csv')

In [5]:
import pandas as pd

# 假设 df 和 df2 顺序完全一致
# 直接将 df2 中的 'epitope_smi' 列添加到 df 中
df['epitope_smi'] = df2['epitope_smi'].values

# 查看合并后的 df
print(df)


                    x              y  similarity_to_own_cluster  \
0      CASSFVGGTDTQYF      TTLPVNVAF                   0.033879   
1     CASSQDLRGDNEQFF  LSDDAVVCFNSTY                  -0.287606   
2      CASSLSGLNTEAFF      RTLNAWVKV                  -0.321876   
3       CASSTPGDNEQFF     AVFDRKSDAK                   0.500634   
4     CASSLTSGSDNEQFF      KLSYGIATV                   0.062706   
...               ...            ...                        ...   
9431   CASSLEWGGETQYF      RLDKVEAEV                   0.634236   
9432     CASSAREGKLFF      FLPRVFSAV                   0.227013   
9433   CASRPSGGAETQYF    HPVGEADYFEY                   0.555592   
9434     CASSRHTGELFF     KRWIILGLNK                   0.882961   
9435     CASSLGTTEAFF      RLDKVEAEV                  -0.281755   

      max_similarity  min_similarity  avg_similarity  median_similarity  \
0           0.961727       -0.236894        0.225555           0.180564   
1           0.950048       -0.496050        0

In [6]:
X = df.drop(columns=['x', 'label'])
X

,y,similarity_to_own_cluster,max_similarity,min_similarity,avg_similarity,median_similarity,std_similarity,skewness_similarity,kurtosis_similarity,rank_position,epitope_smi
0,TTLPVNVAF,0.033879,0.961727,-0.236894,0.225555,0.180564,0.270333,0.712941,0.011054,52,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)[C@@H]...
1,LSDDAVVCFNSTY,-0.287606,0.950048,-0.496050,0.121017,0.088571,0.339725,0.441316,-0.435056,26,CC(C)C[C@H](N)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(...
2,RTLNAWVKV,-0.321876,0.942863,-0.442244,0.103192,0.075691,0.336582,0.546719,-0.312440,18,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)CCCNC(...
3,AVFDRKSDAK,0.500634,0.909818,-0.242934,0.246745,0.216040,0.264842,0.411334,-0.463103,158,CC(C)[C@H](NC(=O)[C@H](C)N)C(=O)N[C@@H](Cc1ccc...
4,KLSYGIATV,0.062706,0.862396,-0.375197,0.153515,0.107316,0.298526,0.391524,-0.762041,85,CC[C@H](C)[C@H](NC(=O)CNC(=O)[C@H](Cc1ccc(O)cc...
...,...,...,...,...,...,...,...,...,...,...,...
9431,RLDKVEAEV,0.634236,0.949677,-0.358064,0.164072,0.108038,0.302528,0.717219,-0.064478,174,CC(C)C[C@H](NC(=O)[C@@H](N)CCCNC(=N)N)C(=O)N[C...
9432,FLPRVFSAV,0.227013,0.922438,-0.468003,0.130270,0.071077,0.362090,0.449400,-0.725087,123,CC(C)C[C@H](NC(=O)[C@@H](N)Cc1ccccc1)C(=O)N1CC...
9433,HPVGEADYFEY,0.555592,0.971113,-0.313434,0.141064,0.015659,0.346441,0.719641,-0.714006,159,CC(C)[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](N)Cc1...
9434,KRWIILGLNK,0.882961,0.897994,-0.482312,0.146344,0.173266,0.322539,-0.091331,-0.787925,191,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1c[nH]c2ccccc12)...


In [7]:
non_numeric_columns = X.select_dtypes(include=['object']).columns
if 'y' in non_numeric_columns:
    # 对 'y' 列进行独热编码
    encoder = OneHotEncoder(sparse_output=False, drop='first')
    y_encoded = encoder.fit_transform(df[['y']])
    y_encoded_df = pd.DataFrame(y_encoded, columns=encoder.get_feature_names_out(['y']))

    # 将独热编码的 'y' 列与其他特征合并，并删除原始的 'y' 列
    X = X.drop(columns=['y'])
    X = pd.concat([X, y_encoded_df], axis=1)
X

,similarity_to_own_cluster,max_similarity,min_similarity,avg_similarity,median_similarity,std_similarity,skewness_similarity,kurtosis_similarity,rank_position,epitope_smi,...,y_YFPLQSYGF,y_YIFFASFYY,y_YLDAYNMMI,y_YLKLTDNVYIK,y_YLNTLTLAV,y_YLQPRTFLL,y_YSEHPTFTSQY,y_YVFCTVNAL,y_YVLDHLIVV,y_YYRRATRRIR
0,0.033879,0.961727,-0.236894,0.225555,0.180564,0.270333,0.712941,0.011054,52,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)[C@@H]...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.287606,0.950048,-0.496050,0.121017,0.088571,0.339725,0.441316,-0.435056,26,CC(C)C[C@H](N)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.321876,0.942863,-0.442244,0.103192,0.075691,0.336582,0.546719,-0.312440,18,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)CCCNC(...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.500634,0.909818,-0.242934,0.246745,0.216040,0.264842,0.411334,-0.463103,158,CC(C)[C@H](NC(=O)[C@H](C)N)C(=O)N[C@@H](Cc1ccc...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.062706,0.862396,-0.375197,0.153515,0.107316,0.298526,0.391524,-0.762041,85,CC[C@H](C)[C@H](NC(=O)CNC(=O)[C@H](Cc1ccc(O)cc...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9431,0.634236,0.949677,-0.358064,0.164072,0.108038,0.302528,0.717219,-0.064478,174,CC(C)C[C@H](NC(=O)[C@@H](N)CCCNC(=N)N)C(=O)N[C...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9432,0.227013,0.922438,-0.468003,0.130270,0.071077,0.362090,0.449400,-0.725087,123,CC(C)C[C@H](NC(=O)[C@@H](N)Cc1ccccc1)C(=O)N1CC...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9433,0.555592,0.971113,-0.313434,0.141064,0.015659,0.346441,0.719641,-0.714006,159,CC(C)[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](N)Cc1...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9434,0.882961,0.897994,-0.482312,0.146344,0.173266,0.322539,-0.091331,-0.787925,191,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1c[nH]c2ccccc12)...,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
!pip install rdkit-pypi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 12.3 MB/s eta 0:00:00


In [9]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return pd.Series([None] * 5)  # 如果 SMILES 无效，返回空值
    features = {
        'mol_weight': Descriptors.MolWt(mol),
        'logP': Descriptors.MolLogP(mol),
        'tpsa': Descriptors.TPSA(mol),
        'num_h_donors': Descriptors.NumHDonors(mol),
        'num_h_acceptors': Descriptors.NumHAcceptors(mol)
    }
    return pd.Series(features)

# 应用函数到 'epitope_smi' 列并添加到 X DataFrame
X_features = X['epitope_smi'].apply(extract_features)
X = pd.concat([X, X_features], axis=1)
X

,similarity_to_own_cluster,max_similarity,min_similarity,avg_similarity,median_similarity,std_similarity,skewness_similarity,kurtosis_similarity,rank_position,epitope_smi,...,y_YLQPRTFLL,y_YSEHPTFTSQY,y_YVFCTVNAL,y_YVLDHLIVV,y_YYRRATRRIR,mol_weight,logP,tpsa,num_h_donors,num_h_acceptors
0,0.033879,0.961727,-0.236894,0.225555,0.180564,0.270333,0.712941,0.011054,52,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)[C@@H]...,...,0.0,0.0,0.0,0.0,0.0,961.128,-2.95940,370.88,12.0,13.0
1,-0.287606,0.950048,-0.496050,0.121017,0.088571,0.339725,0.441316,-0.435056,26,CC(C)C[C@H](N)C(=O)N[C@@H](CO)C(=O)N[C@@H](CC(...,...,0.0,0.0,0.0,0.0,0.0,1433.560,-7.09510,611.13,22.0,22.0
2,-0.321876,0.942863,-0.442244,0.103192,0.075691,0.336582,0.546719,-0.312440,18,CC(C)C[C@H](NC(=O)[C@@H](NC(=O)[C@@H](N)CCCNC(...,...,0.0,0.0,0.0,0.0,0.0,1086.307,-2.97283,463.15,17.0,14.0
3,0.500634,0.909818,-0.242934,0.246745,0.216040,0.264842,0.411334,-0.463103,158,CC(C)[C@H](NC(=O)[C@H](C)N)C(=O)N[C@@H](Cc1ccc...,...,0.0,0.0,0.0,0.0,0.0,1136.276,-5.83703,533.99,19.0,17.0
4,0.062706,0.862396,-0.375197,0.153515,0.107316,0.298526,0.391524,-0.762041,85,CC[C@H](C)[C@H](NC(=O)CNC(=O)[C@H](Cc1ccc(O)cc...,...,0.0,0.0,0.0,0.0,0.0,951.133,-2.87340,382.83,14.0,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9431,0.634236,0.949677,-0.358064,0.164072,0.108038,0.302528,0.717219,-0.064478,174,CC(C)C[C@H](NC(=O)[C@@H](N)CCCNC(=N)N)C(=O)N[C...,...,0.0,0.0,0.0,0.0,0.0,1058.202,-3.74873,495.94,17.0,15.0
9432,0.227013,0.922438,-0.468003,0.130270,0.071077,0.362090,0.449400,-0.725087,123,CC(C)C[C@H](NC(=O)[C@@H](N)Cc1ccccc1)C(=O)N1CC...,...,0.0,0.0,0.0,0.0,0.0,1035.258,-1.09813,369.46,13.0,12.0
9433,0.555592,0.971113,-0.313434,0.141064,0.015659,0.346441,0.719641,-0.714006,159,CC(C)[C@H](NC(=O)[C@@H]1CCCN1C(=O)[C@@H](N)Cc1...,...,0.0,0.0,0.0,0.0,0.0,1326.385,-2.63080,526.57,17.0,18.0
9434,0.882961,0.897994,-0.482312,0.146344,0.173266,0.322539,-0.091331,-0.787925,191,CC[C@H](C)[C@H](NC(=O)[C@H](Cc1c[nH]c2ccccc12)...,...,0.0,0.0,0.0,0.0,0.0,1240.564,-1.30203,498.04,18.0,15.0


In [10]:
X = X.apply(pd.to_numeric, errors='coerce')

In [11]:
X = X.drop(columns=['epitope_smi'])

In [12]:
# 目标标签
y = df['label']

# 3. 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. 初始化并训练 XGBoost 分类模型
model = XGBClassifier(
    n_estimators=100,  # 基模型数量
    max_depth=5,       # 树的最大深度
    learning_rate=0.1, # 学习率
    random_state=42    # 随机种子
)
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [13]:
y_pred = model.predict(X_test)

# 6. 评估模型性能
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_report(y_test, y_pred))

Accuracy: 0.7314618644067796
Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.72      0.73       959
           1       0.72      0.74      0.73       929

    accuracy                           0.73      1888
   macro avg       0.73      0.73      0.73      1888
weighted avg       0.73      0.73      0.73      1888



In [15]:
from sklearn.metrics import average_precision_score, precision_recall_curve, auc, roc_auc_score, f1_score

# 假设 y_test 为真实标签，y_pred 为预测标签，y_prob 为预测概率

# 1. AUPRC (PR曲线下的面积)
auprc = average_precision_score(y_test, y_pred)
print(f"AUPRC (average_precision_score): {auprc}")

# 2. AUROC (ROC曲线下的面积)
auroc = roc_auc_score(y_test, y_pred)
print(f"AUROC (roc_auc_score): {auroc}")

# 3. F1-score
f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1}")


AUPRC (average_precision_score): 0.6615436225828829
AUROC (roc_auc_score): 0.7316381771018655
F1-score: 0.7313195548489666


In [ ]:
# !pip install PyTDC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.8/146.8 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 2.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of s3fs to determine which version is compatible with other requirements. This could take a while.
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.1/315.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.4 MB/s eta